In [1]:
import numpy as np
import tensorflow as tf

import sys
sys.path.append(r'D:\Users\Madhouse\tensorflow\yobiface\src')
import utils

import os
from os.path import join as pjoin
import copy
import shutil
import pickle
import time as t
 
import cv2  
import detect_face  
import time 
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

d:\Users\Madhouse\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.7.0


In [2]:
def load_all_pics(path):
    data = {}
    pics_ctr = 0
    files= os.listdir(path)
    for file in files: #遍历文件夹 
        if not os.path.isdir(file): #判断是否是文件夹，不是文件夹才打开
        #print(file)
            person_dir = pjoin(path, file)
            if(file.find(".jpg") > 0):
                guy=file.split("/")[-1][:-4]
                #print("guy[%d]=%s"%(pics_ctr,guy))
                curr_pics = [utils.preproc(cv2.imread(person_dir))]
                data[guy] = curr_pics
                pics_ctr += len(curr_pics)
    return data, pics_ctr

def load_emb(path):
    pickle_file = open(path, 'rb')
    my_dict = pickle.load(pickle_file)
    # 一定要注意 要写关闭文件
    pickle_file.close()
    print("read dict from file is finished")
    return my_dict

def getTime():
    return t.strftime('%Y-%m-%d %H:%M:%S',t.localtime())

def weigth_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def getBatch1(arr_x, arr_y, iter_cnt, batch_size=100):
    #assert (Temperature >= 0),"Colder than absolute zero!"
    assert(len(arr_x)==len(arr_y)),"lens of arr_x and arr_y are not equal..."
    length=len(arr_x)
    maxcnt=length//batch_size + 1
    iter_cnt=iter_cnt%maxcnt
    start=batch_size*(iter_cnt)
    end=min(batch_size*(iter_cnt+1),length)
    return arr_x[start:end],arr_y[start:end]

def getBatch(data_set, iter_cnt, batch_size=100):
    length=len(data_set)
    maxcnt=length//batch_size + 1
    iter_cnt=iter_cnt%maxcnt
    start=batch_size*(iter_cnt)
    end=min(batch_size*(iter_cnt+1),length)
    return data_set[start:end][:,1:],data_set[start:end][:,0]

def nextBatch(data_set, iter_cnt, batch_size=100):
    length=len(data_set)
    maxcnt=length//batch_size + 1
    iter_cnt=iter_cnt%maxcnt
    start=batch_size*(iter_cnt)
    end=min(batch_size*(iter_cnt+1),length)
    return data_set[start:end][:,1:],np.expand_dims(data_set[start:end][:,0], axis=1)

def fill_feed_dictCNN(data_set, type, images_pl, labels_pl, keep_prob, iter_cnt, batch_size=100):
    """Fills the feed_dict for training the given step.
    A feed_dict takes the form of:
    feed_dict = {
        <placeholder>: <tensor of values to be passed for placeholder>,
        ....
    }
    Args:
      data_set: The set of images and labels, from input_data.read_data_sets()
      images_pl: The images placeholder, from placeholder_inputs().
      labels_pl: The labels placeholder, from placeholder_inputs().
    Returns:
      feed_dict: The feed dictionary mapping from placeholders to values.
    """
    # Create the feed_dict for the placeholders filled with the next
    # `batch size ` examples.
    #batch = data_set.next_batch(FLAGS.batch_size, FLAGS.fake_data)
    x,y=getBatch(data_set, iter_cnt, batch_size)
    kp = 0.5
    if type.find("train") == -1:
        kp = 1.0 
    feed_dict = {
        images_pl: x,
        labels_pl: y,
      keep_prob: kp
    }
    return feed_dict

In [3]:
emb_arr = load_emb('emb_arr.pkl')
print('emb_arr length = %d'%len(emb_arr))
emb_map = load_emb('emb_map.pkl')
print('emb_map length = %d'%len(emb_map))

x,y=nextBatch(emb_arr, 0, 500)
print(x.shape, y.shape)

read dict from file is finished
emb_arr length = 13268
read dict from file is finished
emb_map length = 5761
(500, 512) (500, 1)


In [4]:
#face detection parameters  
minsize = 20 # minimum size of face  
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold  
factor = 0.709 # scale factor 
print('Creating networks and loading parameters')  
with tf.Graph().as_default():  
    sess = tf.Session() 
    with sess.as_default():  
        pnet, rnet, onet = detect_face.create_mtcnn(sess, './mtcnn_model/')
        saver = tf.train.import_meta_graph(r'D:\Users\Madhouse\tensorflow\download\20180402-114759\model-20180402-114759.meta')
        saver.restore(sess, r'D:\Users\Madhouse\tensorflow\download\20180402-114759\model-20180402-114759.ckpt-275')
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
print("model restore finished.....")

Creating networks and loading parameters
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from D:\Users\Madhouse\tensorflow\download\20180402-114759\model-20180402-114759.ckpt-275
model restore finished.....


In [ ]:
class_nbr=len(emb_map)
x= tf.placeholder("float", shape=[None, 512])
y_ = tf.placeholder("float", shape=[None, 1], name="y")

A = tf.Variable(tf.random_normal(shape=[512, 1]))
b = tf.Variable(tf.random_normal(shape=[1, 1]))

# 定义线性模型
output = tf.subtract(tf.matmul(x, A), b)

# Declare vector L2 'norm' function squared
l2_norm = tf.reduce_sum(tf.square(A))

# Loss = max(0, 1-pred*actual) + alpha * L2_norm(A)^2
alpha = tf.constant([0.01])
classification_term = tf.reduce_mean(tf.maximum(0., tf.subtract(1., tf.multiply(output, y_))))
loss = tf.add(classification_term, tf.multiply(alpha, l2_norm))

prediction=tf.sign(output)
accuracy=tf.reduce_mean(tf.cast(tf.equal(prediction,y_),tf.float32))

optimizer = tf.train.GradientDescentOptimizer(0.01)
train_op = optimizer.minimize(loss)

se = tf.Session()
se.run(tf.global_variables_initializer())
with se.as_default():
    batch_size=500
    for i in range(10000):
        #feed_dict=fill_feed_dictCNN(emb_arr, "train", x, y_, keep_prob, i, batch_size)
        batch=nextBatch(emb_arr, i, batch_size)
        tt = getTime()
        if i%500 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1]})
            print("[%s] step %d, training accuracy %g"%(tt, i, train_accuracy))
        train_op.run(feed_dict={x:batch[0], y_:batch[1]})
        #train_op.run(feed_dict={x:batch[0], y_:batch[1], keep_prob: kp})